# Formaciones geológicas del Uruguay
## Información extraida de dinamige.gub.uy


In [ ]:
# importing packages 
import seaborn as sns
import pandas as pd 
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os 
import folium
import json

#%matplotlib inline 
%matplotlib qt 

In [ ]:
cwd = os.getcwd()

In [ ]:
#Leemos archivo extraido de atributos
ruta = cwd + '\GeoUyFm.xlsx'

dfFm = pd.read_excel(ruta)
dfFm.shape


In [ ]:
dfFm.to_html

In [ ]:
dfvacios = dfFm.isnull().sum()
dfvacios = dfvacios.to_frame().reset_index()

dfvacios.columns = ['CAMPO', 'VACIOS']



In [ ]:
print(gdf['Unidad'][1785])

In [ ]:
#gdf = gdf.set_crs("EPSG:4326")
#gdf = gdf.set_crs(epsg=4326)

In [ ]:
gdf = gpd.read_file(cwd + "\\Geologia_Uruguay_shp\\Unidades_Geologicas.shp")
estdf = gpd.read_file(cwd + "\\Geologia_Uruguay_shp\\Estructuras.shp")

In [ ]:
estdf = estdf.to_crs(epsg=4326)
estdf.head()

In [ ]:
#gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
gdf.crs
gdf = gdf.to_crs(epsg=4326)
#gdf.set_crs(epsg=4326, inplace=True)
gdf.drop(gdf.loc[gdf['Unidad']=='Actual'].index, inplace=True)
gdf = gdf.sort_values('Unidad',ascending=True,na_position='first')
gdf

#gdf = gdf[(gdf['Unidad']=='Formacion Cerrezuelo')|(gdf['Unidad']=='Formacion Cordobes')]



In [ ]:
#gdf[gdf['Periodo'] == 'Riacico']

In [ ]:
#import pyproj
#pyproj.datadir.get_data_dir()

In [ ]:
#gdf = gpd.read_file(cwd + "\\Geologia_Uruguay_shp\\Unidades_Geologicas.shp")
m = folium.Map(location=[-32.5, -55.3], tiles="cartodbpositron", zoom_start=7)
folium.GeoJson(data=subgdf["geometry"],
              tooltip=subgdf["Eon"][subgdf.index]
              ).add_to(m)


In [ ]:
subgdf.plot(figsize=(6, 6))
plt.show()

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

fire_dict = dict(zip(gdf['FireName'], gdf['Acres']))

tooltip = GeoJsonTooltip(
    fields = ['FireName', 'FireYear', 
              'Acres', 'FireCause'],
    aliases = ['Name', 'Year', 'Acres', 'Cause'],
    #localize = True,
    sticky = True,
    labels = True,
    style = """
    background-color: #F0EFEF;
    border: 2px solid black;
    border-radius: 3px,
    box-shadow: 3px;
    """,
    max_width = 800
)

    


### generación del mapa con las formaciones


In [ ]:
#gdfunidad = gdf.groupby('Unidad')
#listUnidades = gdf.groupby('Unidad').size().to_frame().reset_index().iloc[:,0].to_list()
listUnidades = gdf['Unidad'].to_list()

listColores =[]
ud =''

for unidad in listUnidades:
    if ud != unidad:
        color = np.random.randint(16, 256, size=3)
        color = [str(hex(i))[2:] for i in color]
        color = '#'+''.join(color).upper()
        ud = unidad        
    listColores.append(color)
    
listColores = listColores*2
    


In [ ]:
def map_colors(feature):
    color = listColores.pop(0)
    return {
        'fillColor': color, 
        'color': 'black',
        'weight': 0.25,
        'fillOpacity': 0.3        
}

In [ ]:
def putpopup(fm,era,des,cod):
    salida = '''
    <h4>{fm}</h4>
    <h5>{era}</h5>
    <p>{des}...<a href="https://rubensdmp.github.io/GeoUy/Formaciones.html#{cod}" target="_blank">más</a></p>
    '''.format(fm=fm,era=era,des=des[:70],cod=cod)
    return salida
    

In [ ]:
#m = folium.Map(location=[-32.5, -55.3], zoom_start=7, tiles='openstreetmap')
m = folium.Map(location=[-32.5, -55.3], zoom_start=7.2, tiles='Stamen Terrain')
               
##,crs='EPSG4326')

listUnidades = gdf['Unidad'].to_list()

listColores =[]
ud =''

for unidad in listUnidades:
    if ud != unidad:
        color = np.random.randint(16, 256, size=3)
        color = [str(hex(i))[2:] for i in color]
        color = '#'+''.join(color).upper()
        ud = unidad        
    listColores.append(color)
    
listColores = listColores*2


iframe = folium.IFrame('html',
                       width=100,
                       height=100)

#popup = folium.Popup(iframe,
#                     max_width=100)



for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=map_colors)
    folium.Popup(putpopup(r['Unidad'],r['Periodo'],r['Descripcio'],r['Codigo']),
                 sticky=False,parse_html=False,max_width=100
                 ).add_to(geo_j)
    geo_j.add_to(m)
    
m


In [ ]:
gdf.shape

In [ ]:
for _, r in estdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda x: {'fillColor': 'black'})
    folium.Popup(r['Nombre']).add_to(geo_j)
    geo_j.add_to(m)
m

In [ ]:
m.save("GeoUy.html")

In [ ]:
for x in gdf.index:
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    color = '#'+''.join(color).upper()
    gdf.at[x, 'color'] = color

def style(feature):
        return {
            'fillColor': feature['properties']['color'],
            'color': feature['properties']['color'],
            'weight': 1
        }

folium.GeoJson(data=gdf, style_function=style).add_to(m)

In [ ]:
folium.Marker(
    location=[-32.5, -55.3],
    popup='Kumpula Campus',
    tooltip='poronga negra',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(m)


m

In [ ]:
GeoUy_map = r'd:\Users\d836148\Dropbox\DataScience\GitHub\GeoPython\GeoUy\GeoUy.geojson' # geojson file

displat = -32.5
displong = -55.3

# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
basemap = folium.Map(location=[displat, displong], zoom_start=7, tiles='Stamen Terrain')


In [ ]:
GeoUy_map = r'd:\Users\d836148\Dropbox\DataScience\GitHub\GeoPython\GeoUy\GeoUy.geojson' # geojson file

# generate choropleth map 
CoroUy = folium.Choropleth(
    geo_data=GeoUy_map,
    data=gdf["geometry"],
    columns=dfvacios.columns,
    key_on='feature.properties.Eon',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    highlight=True,
    legend_name='Formaciones Geológicas Uy',
    smooth_factor=0,
    #tooltip=(by_count[i] for i in by_count['dep'])
    ).add_to(basemap)


# add labels indicating the name of the community
#style_function = "font-size: 15px; font-weight: bold"
#chocountry.geojson.add_child(
#    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False),
#    folium.LayerControl().add_to(world_map))

#CoroUy.geojson.add_child(
#    folium.features.GeoJsonTooltip(
#    fields=['Eon', 'Era'],
#    aliases=['Eon', 'Era'],
#    style=('background-color: grey; color: white;')
#    )
#)

# display map
basemap